<a href="https://colab.research.google.com/github/babypotatotang/Developing-CurrentForecastIndex-for-ShippingIndustry/blob/main/1.%20Analysis%20Sentimental/(2)%20Modeling/(3)%20Modeling%20Sentiment%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import files 

# 데이터 load 및 기본 스펙 확인 : train_data, test_data

In [ ]:
train_data=pd.read_csv(filename),usecols=['Sentence','Label'],encoding='cp949')
test_data=pd.read_csv(filename,usecols=['Sentence','Label'],encoding='cp949')
\

In [ ]:
train_data['Sentence'].nunique(), train_data['Label'].nunique() 

In [ ]:
test_data['Sentence'].nunique(), test_data['Label'].nunique()

In [ ]:
train_data['Label']=train_data['Label'].astype(int)
train_data['Label']=train_data['Label'].apply(lambda x: 0 if x==-1 else x) #부정 레이블 0으로 설정

test_data['Label']=test_data['Label'].astype(int)
test_data['Label']=test_data['Label'].apply(lambda x: 0 if x==-1 else x) #부정 레이블 0으로 설정

In [ ]:
train_data.drop_duplicates(subset = ['Sentence'], inplace=True) # 중복 제거
train_data['Sentence'] = train_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #한글과 공백을 제외하고 모두 제거
train_data['Sentence'].replace('', np.nan, inplace=True)#값이 없는 경우 Null 값으로 변경
train_data.dropna(how='any',inplace=True) # Null 값 제거

print('전처리 후 훈련용 샘플의 개수 :',len(train_data))
print('null값 존재 유무: ',train_data.isnull().values.any())

In [ ]:
test_data.drop_duplicates(subset = ['Sentence'], inplace=True) # 중복 제거
test_data['Sentence'] = test_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #한글과 공백을 제외하고 모두 제거
test_data['Sentence'].replace('', np.nan, inplace=True)#값이 없는 경우 Null 값으로 변경
test_data.dropna(how='any',inplace=True) # Null 값 제거

print('훈련용 데이터 개수: ',len(test_data))
print('null값 존재 유무: ',test_data.isnull().values.any())

# 데이터 전처리: sentence 토큰화

In [ ]:
# #tokenized를 list로 변경
mecab=Mecab()
stopwords = ['했','있','으로','로','것','씨','말','도', '는', '다', '의', '가', '이', '은','수','에서'
             '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '겜', '되', '음', '면']

In [ ]:
# [Train Data] 전처리
train_data['tokenized']=train_data['Sentence'].apply(mecab.morphs) #Sentence 내용을 morphs로 형태소 분석(type: list)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords]) #해당 열의 값 중 stopword에 해당하는 값 지우기
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if len(item)>1]) #길이 2이상만 저장

In [ ]:
# [Test Data] 전처리
test_data['tokenized']=test_data['Sentence'].apply(mecab.morphs) #Sentence 내용을 morphs로 형태소 분석(type: list)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords]) #해당 열의 값 중 stopword에 해당하는 값 지우기
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if len(item)>1]) #길이 2이상만 저장

#데이터 전처리: common word 제거

In [ ]:
def len_by_sentimental(df):
    positive_words = np.hstack(df[df.Label == 1]['tokenized'].values)
    positive_word_count = Counter(positive_words)

    negative_words = np.hstack(df[df.Label == 0]['tokenized'].values)
    negative_word_count = Counter(negative_words)

    print('긍정 단어 길이: ',len(positive_word_count))
    print('부정 단어 길이: ',len(negative_word_count))

    return negative_word_count,positive_word_count

In [ ]:
def common_word_list(common_num,neg,pos):
    negative_word=[]; positive_word=[]
    n_list=neg.most_common(common_num); p_list=pos.most_common(common_num)

    for i in range(common_num):
        negative_word.append(n_list[i][0])
        positive_word.append(p_list[i][0])

    common_list=list(set(negative_word) & set(positive_word))

    print(common_list)
    print('common_list 길이', len(common_list))

    return common_list

In [ ]:
#[Train Data]
train_neg,train_pos=len_by_sentimental(train_data) #common_word 산출 범위(num range) 결정
common_list=common_word_list(20,train_neg,train_pos)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in common_list]) #해당 열의 값 중 stopword에 해당하는 값 지우기

In [ ]:
#[Test Data]
test_neg,test_pos=len_by_sentimental(test_data) #common_word 산출 범위(num range) 결정
common_list=common_word_list(20,test_neg,test_pos)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in common_list]) #해당 열의 값 중 stopword에 해당하는 값 지우기

In [ ]:
train_data.to_csv(filename)
test_data.to_csv(filename)

In [ ]:
negative_words = np.hstack(train_data[train_data.Label == 0]['tokenized'].values)
negative_word_count = Counter(negative_words)
print(negative_word_count.most_common(20))

# 전처리 후 그래프 확인

In [ ]:
print('전처리 후 훈련용 샘플의 개수 :',len(train_data))
print('null값 존재 유무: ',train_data.isnull().values.any())

In [ ]:
print('전처리 후 훈련용 샘플의 개수 :',len(test_data))
print('null값 존재 유무: ',test_data.isnull().values.any())

In [ ]:
X_train = train_data['tokenized'].values
y_train = train_data['Label'].values
X_test= test_data['tokenized'].values
y_test = test_data['Label'].values

# (Sentence) 정수 인코딩

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) #문자데이터를 입력받아 리스트 형태로 변환, 각 단어에 index 부여

In [ ]:
threshold = 6 #단어의 등장 빈도수에 대한 임계치 
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

#사용되는 단어 집합의 크기 
vocab_size = total_cnt - rare_cnt + 2 #0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') #새 vocab_size로 tokenizer 새로 설정
tokenizer.fit_on_texts(X_train)

#X_train, X_test의 데이터를 넣어서 인코딩 
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# (Sentence) 패딩

In [ ]:
print('뉴스의 최대 길이 :',max(len(l) for l in X_train)) 
print('뉴스의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
# 희귀 단어의 개수만큼 제거하는 함수, max_len은 리뷰의 최대 및 평균 길이를 보고 비교해서 설정
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 1000
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# 머신러닝 모델링

In [ ]:
import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

In [ ]:
loaded_model = load_model('best_model.h5')
print("테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
model.save('best_model.h5')

In [ ]:
files.download('best_model.h5')